In [215]:
# import packages for transformations
import sqlite3
import pandas as pd
from pathlib import Path


# Check connection to markdown


In [216]:
#1. path to this script
current_dir = Path.cwd()

#2. Path to project root
project_root = current_dir.parent

#3. Go to database folder
db_path = project_root / "database" / "dataverse_complete.db"

#4. define the connection to the database
try:
    conn = sqlite3.connect(str(db_path))
    print("✅ Connection Successful!")
    print(f"Connected to: {db_path}")
except Exception as e:
    print(f"❌ Still failing. Looking at: {db_path}")
    print(f"Error: {e}")



✅ Connection Successful!
Connected to: /Users/lottesavelberg/Documents/Akvo/IGH/igh-data-transform/src/igh_data_transform/database/dataverse_complete.db


In [217]:
#if the connection was succesful, open it again
conn = sqlite3.connect(str(db_path))

### Set link to Candidates table

In [218]:
#1. define the candidates table
table_name = "vin_candidates"
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# see the first rows
print(f"Table' {table_name} was successfully loaded!")
df.head()

Table' vin_candidates was successfully loaded!


,row_id,vin_name,new_sbereviewstatus,new_potentialforacceleratedorconditionalregulator,vin_stringentregulatoryauthoritysraapprovalda,crc8b_updatedforipps20,vin_targettoxinclass,vin_includeinevgendatabase,new_platform,new_personslivingwithhiv,...,vin_approvingauthority,_createdonbehalfby_value,_vin_clinicalusestatus_value,new_includeinpipeline2021,_vin_routeofadministration_value,new_rdstage,json_response,sync_time,valid_from,valid_to
0,1,DPP Fever Panel II Asia IgM,NaN,None,None,862890000.0,NaN,909670000.0,Lateral Flow Assay,NaN,...,None,None,None,NaN,None,None,"{""_createdby_value"": ""fad346cd-d639-f011-b4cb-...",2026-01-09T12:00:11.613384+00:00,2025-12-17T17:48:00Z,None
1,2,TRURAPID MPXV Ag Test,NaN,None,None,862890000.0,NaN,909670000.0,Lateral Flow Assay,NaN,...,None,None,None,NaN,None,None,"{""_createdby_value"": ""fad346cd-d639-f011-b4cb-...",2026-01-09T12:00:11.755163+00:00,2025-12-17T17:48:00Z,None
2,3,NABIT Mpox Test,NaN,None,None,862890000.0,NaN,909670000.0,qRT-PCR,NaN,...,None,None,None,NaN,None,None,"{""_createdby_value"": ""fad346cd-d639-f011-b4cb-...",2026-01-09T12:00:11.776114+00:00,2025-12-17T17:48:00Z,None
3,4,MPV Ag Rapid Test,NaN,None,None,862890000.0,NaN,909670000.0,Lateral Flow Assay,NaN,...,None,None,None,NaN,None,None,"{""_createdby_value"": ""fad346cd-d639-f011-b4cb-...",2026-01-09T12:00:11.794741+00:00,2025-12-17T17:48:00Z,None
4,5,SGTi-flex Mpox Ag,NaN,None,None,862890000.0,NaN,909670000.0,ELISA/EIA,NaN,...,None,None,None,NaN,None,None,"{""_createdby_value"": ""fad346cd-d639-f011-b4cb-...",2026-01-09T12:00:11.805713+00:00,2025-12-17T17:48:00Z,None


### Execute transformation

In [219]:
#1. Calculate stats of each column
stats = []
for col in df.columns:
   stats.append({
       'column_name': col,
       'unique_values': df[col].nunique(),
       'empty_cells': df[col].isnull().sum(),
       'data_type': df[col].dtypes
   })

#2. Create data frame for the stats
df_stats = pd.DataFrame(stats)

#3. Sort stats data frame by number of unique values to identify index column
df_stats = df_stats.sort_values(by='unique_values', ascending=False)

#4. Print the results
print("Column Quality Summary:")
display(df_stats)

Column Quality Summary:


,column_name,unique_values,empty_cells,data_type
0,row_id,9383,0,int64
190,vin_candidateid,9383,0,object
43,versionnumber,9383,0,int64
244,sync_time,9383,0,object
243,json_response,9383,0,object
...,...,...,...,...
89,new_includeinpipeline2025,0,9383,object
158,new_rdstage2021,0,9383,object
44,new_knownfunders2021,0,9383,object
61,_owningteam_value,0,9383,object


In [220]:
# set index column
df = df.set_index('versionnumber')

In [221]:
#drop all columns with only empty cells
#1. Identify all column with only empty cells
empty_cols = df.columns[df.isnull().all()].tolist()

#2. Print empty colls
if not empty_cols:
    print("No columns are empty")
else:
    print(f"There are {len(empty_cols)} empty columns")
    for col in empty_cols:
        print(f" - {col}")

# Create a new list excluding 'valid_to'
cols_to_drop = [c for c in empty_cols if c != 'valid_to']

# Drop the filtered list
df_transformed = df.drop(columns=cols_to_drop)




There are 20 empty columns
 - crc8b_sourceofipps20update
 - new_knownfunders2021
 - overriddencreatedon
 - _owningteam_value
 - new_developers2021
 - new_includeinpipeline2025
 - utcconversiontimezonecode
 - _new_archetype_simple_value
 - new_developers2025
 - vin_2019notes
 - new_knownfunders2025
 - new_rdstage2021
 - new_vin_snakespeciesagainst
 - _modifiedonbehalfby_value
 - vin_sbereviewcompleted
 - vin_typehidden
 - _createdonbehalfby_value
 - _vin_routeofadministration_value
 - new_rdstage
 - valid_to


In [222]:
len(cols_to_drop)

19

In [223]:
def inspect_column(df, column_name):
    """
    Prints stats, missing data percentage, and top 10 values for a specific column.
    """
    # 1. Check if column exists
    if column_name not in df.columns:
        print(f"❌ Column '{column_name}' not found.")
        return

    print(f"--- Statistics for: {column_name} ---")

    # 2. Basic Stats & Data Type
    unique_vals = df[column_name].nunique()
    empty_cells = df[column_name].isnull().sum()
    filled_cells = df[column_name].count()
    dtype = df[column_name].dtype

    print(f"Unique values: {unique_vals} | Data type: {dtype} | filled cells: {filled_cells}" )

    # 3. Percentage Missing Data
    null_pct = (empty_cells / len(df)) * 100
    print(f"Empty cells: {empty_cells} ({null_pct:.2f}%)")

    #4. Show unique values if they are low (Categorical Data)
    if unique_vals < 20:
        print(f"\n--- All unique values (<20) ---")
        unique_list = df[column_name].dropna().unique()
        print(sorted(unique_list))
    else:
        print(f"\n--- First 10 values ---")
        print(df[column_name].value_counts().head(10))

    print("-" * 40 + "\n")

In [224]:
cols_to_drop_info = ['row_id', 'new_potentialforacceleratedorconditionalregulator',
                                      'crc8b_updatedforipps20', 'vin_stringentregulatoryauthoritysraapprovalda', 'vin_includeinevgendatabase', 'new_platform', 'new_personslivingwithhiv', 'vin_id', 'crc8b_ghtcroireviewstatus', 'vin_2019candidateidnumber', 'new_medicated',
                                      'crc8b_ndpipelinereviewstatus',
                                      'vin_numberofcountrieswithproductapproval_date', 'vin_casnumber',
                                      'new_exportgroup', 'crc8b_ndpipelinereviewdate', 'vin_2019stagepcr',
                                      'new_pipscomments', 'new_adjuvantrequirement', 'new_whreviewdate',
                                      '_modifiedby_value', 'vin_reviewpersonaggregated' ,'vin_iggformatanimalderived', 'new_durationofaction', 'new_reviewstatus', 'vin_fdapregnancylabelingpregnancyrisksummary', '_vin_captype_value',
                                      'new_regionofregistration', 'new_estimateddateofregulatoryfiling',
                                      'vin_regionspecificityaggregated', 'vin_sbereviewdate', 'new_2023includeinevgendatabase', 'vin_currentrdstage', 'vin_evgenreviewcompleted', 'timezoneruleversionnumber', 'new_testformat',
                                     'new_aim1clinicalusestatus', 'new_updatedforaim20',
                                      'new_includeinaim1', '_createdby_value', 'vin_evgenreviewdate', 'crc8b_includeinipps20', 'new_thermostabilityandstorage', 'new_atcclassification', 'vin_meshheadings', 'vin_directactionontoxins',
                                      'vin_duplicateentrycapformorethanonedisease', 'importsequencenumber', 'crc8b_srhreviewdate', 'createdon', 'statuscode', 'crc8b_updatedforndpipeline', 'vin_2019archetype', 'vin_nationalregulatoryauthorityapprovaldate', 'new_tppreviewrequired', 'vin_otherindications', 'vin_includeinwellcomesbedatabase', 'new_includeinportal2025', 'new_aim1archetype',
                                      'new_reviewdateipps30', 'new_chimstudyyesno', 'vin_productiontechniqueandorimmunizationstrat', 'new_ipps30reviewstatus',
                                      'vin_reviewnotes', 'new_profilestatus', 'vin_adisid', 'new_mamedicinesubtype', 'crc8b_includeinghtcroi', 'new_includeinwhpipeline', 'vin_2019pcrpipelineinclusion', 'vin_adisurl',
                                      'new_ctregistrylink2', '_owninguser_value', 'vin_2019disease', 'crc8b_platformtechnologyused',
                                      'new_dateforstream',  '_vin_pipct_value', 'new_reviewdate', 'new_cttitle',
                                      'new_includeinipps30', 'vin_ifyesdoesclinicaltrialevidencepredate2015', 'vin_2019product',
                                      'new_snakespecies_producttestedin', 'new_safetyandreactogenicityprofile', 'new_pesubtype', 'crc8b_descriptionofipps20update', 'new_exportorder', 'new_updatedforipps30', 'vin_syndromicprofiles', 'vin_routeofadministrationaggregated', 'new_safetyreactogenicityprofile',
                                      'vin_isthereevidencethatthecandidatehasbeentes', '_vin_archetype_value', 'new_clinicaltrialgeographicallocation', 'vin_2019candidatename', 'crc8b_srhprofilestatus', 'vin_routeofadministrationaggregated',
                                      'new_safetyreactogenicityprofile', 'new_includeinaim2',
                                      'vin_2019complexitysecondpass', 'crc8b_includeinippscdf', 'new_aim1pcrreviewnotes',
                                      'vin_inactivedevelopmenttype', 'new_impactmodellingurl', 'new_aim1devstatus',
                                      'new_2024includeinpipeline', 'vin_countryspecificityaggregated', 'new_whprofilestatus',  'new_efficacyandclinicalendpoints', 'new_aim1identifier', 'crc8b_originofipps20update', 'vin_2019subdisease', 'vin_stringentregulatoryauthoritysraapproval', 'new_aim1highestrdstage', 'crc8b_includeinndpipeline', '_new_numberofcts_value', 'vin_2019status',  '_ownerid_value', 'crc8b_includeinsrhpipeline',
                                      'new_whereistheresearchlocated',
                                      'vin_ifigformatrecombinantotherpleasespecify',
                                      'crc8b_ipps20reviewstatus', 'vin_iggformatrecombinant', 'new_ctregistrylink3',
                                      'vin_includeinp2imodel',  'new_includeinpipeline2021',
                                        'json_response', 'sync_time', 'vin_igfinalproducttypepreparationifapplicable',
                                      'crc8b_includeinipps10', 'new_aim1currentrdstage', 'new_treatmentregvxschedule',
                                      'new_isthisproductspecifictoaregion', '_owningbusinessunit_value', 'new_interactionwithotherpharmacologicalproducts' ]

In [225]:
len(cols_to_drop_info)

135

In [227]:
# delete insignificant columns

df_transformed = df_transformed.drop(cols_to_drop_info, axis = 1)

In [228]:
# update column name
df_transformed = df_transformed.rename(columns={"vin_name": "candidate_name",
                                                'new_sbereviewstatus': 'sbereviewstatus',
                                                'vin_targettoxinclass' : 'targettoxinclass',
                                                'vin_knownfundersaggregated': 'knownfundersaggregated',
                                                'new_vin_whosnakespeciesriskcatagainst' : 'whosnakespeciesriskcatagainst',
                                                'vin_approvalstatus':  'approvalstatus',
                                                'vin_venomspecificity': 'venomspecificity',
                                                'new_pressuretype': 'pressuretype',
                                                'new_ctregistrylink' : 'ctregistrylink',
                                                'vin_approvedforuseinpregnantorlactatingwomen': 'approvedforuseinpregnantorlactatingwomen',
                                                'new_technologyprinciple' :'technologyprinciple',
                                                'new_ctenddate': 'ctenddate',
                                                'vin_2019developers': '2019developers',
                                                'vin_usfdaapprovaldate': 'usfdaapprovaldate',
                                                'vin_alternativenames': 'alternativenames',
                                                'vin_product' : 'product',
                                                'vin_researchedinpregnantwomenorlactatingwomen': 'researchedinpregnantwomenorlactatingwomen',
                                                'vin_target': 'target',
                                                'vin_emaapprovalstatus': 'emaapprovalstatus',
                                                'vin_inactivedevelopmentreason': 'inactivedevelopmentreason',
                                                'new_2024knownfunders': '2024knownfunders',
                                                'vin_emaapprovaldate':  'emaapprovaldate',
                                                'vin_previouslyidentifiedcandidate' : 'previouslyidentifiedcandidate',
                                                'new_2023knownfundersaggregated': '2023knownfunders',
                                                'crc8b_srhindication': 'WH_indication',
                                                'vin_developersaggregated': 'developersaggregated',
                                                'vin_japanesemhlwapprovalstatus': 'japanesemhlwapprovalstatus',
                                                'vin_typeofpreclinicalresults': 'typeofpreclinicalresults',
                                                'vin_stringentregulatoryauthorityapproval': 'SRA_approvalstatus',
                                                'vin_whoprequalificationdate': 'whoprequalificationdate',
                                                'vin_technologytype': 'technologytype',
                                                'new_snakespeciesagainst' : 'snakespeciesagainst',
                                                'vin_numberofcountrieswithproductapproval': 'numberofcountrieswithproductapproval',
                                                'new_developers2025': 'developers2025',
                                                'vin_indication': 'indication',
                                                'vin_snakespecies': 'snakespecies',
                                                'vin_specifictargettoxinclass': 'specifictargettoxinclass',
                                                'new_ctstartdate': 'ctstartdate',
                                                'vin_chemicalname':'chemicalname',
                                                'vin_specimentype': 'specimentype',
                                                'new_indicationtype': 'indicationtype',
                                                'vin_mechanismofaction': 'mechanismofaction',
                                                'vin_recentupdates': 'recentupdates',
                                                'new_knownfunders2025': 'knownfunders2025',
                                                'vin_usfdaapprovalstatus': 'usfdaapprovalstatus',
                                                'vin_preclinicalresultsstatus':'preclinicalresultsstatus',
                                                'new_2023developersaggregated': '2023developersaggregated',
                                                'new_anticipatedsranraandwhopqstrategy': 'anticipatedwhostrategy',
                                                'vin_whoprequalification': 'whoprequalification',
                                                'new_2024developersaggregated': '2024developersaggregated',
                                                'vin_healthcarefacilitylevel': 'healthcarefacilitylevel',
                                                'new_2023currentrdstage': '2023currentrdstage',
                                                '_vin_mainproduct_value': 'mainproduct_value',
                                                '_vin_subproduct_value': 'subproduct_value',
                                                '_vin_secondarydisease_value': 'secondarydisease_value',
                                                'new_snakespeciesproductderivedfrom': 'snakespeciesproductderivedfrom',
                                                'vin_keyfeatureschallenges': 'keyfeatureschallenges',
                                                'new_agespecific': 'agespecific',
                                                'vin_candidateid': 'candidateid',
                                                'new_whoimmunizingspecies': 'whoimmunizingspecies',
                                                'vin_japanesemhlwapprovaldate': 'japanesemhlwapprovaldate',
                                                'vin_developmentstatus': 'developmentstatus',
                                                'vin_snakefamily': 'snakefamily',
                                                'new_2024currentrdstage': '2024rdstage',
                                                'vin_otherstringentregulatoryauthoritydate': 'otherSRAdate',
                                                'vin_nationalregulatoryauthorityapprovalstatus': 'NRAapprovalstatus',
                                                'new_2024developmentstatus': '2024developmentstatus', '_vin_disease_value': 'diseasevalue',
                                                'new_includeinpipeline': 'includeinpipeline',
                                                'vin_whosnakespeciesriskcategory': 'whosnakespeciesriskcategory',
                                                'vin_preclinicalresultssource': 'preclinicalresultssource',
                                                'vin_countrieswhereproductisapprovedaggregated': 'countries_product_approved',
                                                'new_whoparaspecificityspecies': 'whoparaspecificityspecies',
                                                'vin_thermostability': 'thermostability',
                                                'new_peseverity': 'peseverity',
                                                'new_2023developmentstatus': '2023developmentstatus',
                                                '_vin_currentrndstage_value': 'rdstage_value',
                                                'vin_approvingauthority': 'approvingauthority',
                                                '_vin_clinicalusestatus_value': 'clinicalusestatus_value'
                                                })



In [229]:
df_transformed.shape

(9383, 94)

In [230]:
# Update pressure type categorical values
df_transformed['pressuretype'] = df_transformed['pressuretype'].replace({'Negative pressure ' : 'Negative pressure', 'Positive pressure ' : 'Positive pressure', 'Not applicable ': 'N/A'})

In [231]:
# Update product categorical values
# This will list every unique status and how many rows have it
status_counts = df[('vin_product')].value_counts()

print(status_counts)

df_transformed['product'] = df_transformed['product'].replace({'Dietary supplement' : 'Dietary supplements',                                                                    'Diagnostic' :'Diagnostics',
                                                                'Drug':'Drugs',
                                                               'Functional foods' :'Dietary supplements',
                                                               'Microbial interventions':'Microbicides',
                                                               'Chemical vector control products' : 'VCP',
                                                               'Biological vector control products' : 'VCP',
                                                               'Vector control products Reservoir targeted vaccines': 'VCP',
                                                                'Vector control products' :'VCP',
                                                               'Reservoir targeted vaccines' : 'Vaccines'
                                                               }
)



vin_product
Diagnostics                                            3817
Drugs                                                  2309
Vaccines                                               1623
Biologics                                               861
Dietary supplements                                     237
Chemical vector control products                        107
Devices                                                  94
Microbicides                                             26
Biological vector control products                       11
Diagnostic                                                6
Vector control products Reservoir targeted vaccines       4
Drug                                                      2
Vector control products                                   2
Functional foods                                          2
Reservoir targeted vaccines                               2
Microbial interventions                                   1
Dietary supplement          

In [233]:
#update categories of pre clinical result status

status_counts = df[('vin_preclinicalresultsstatus')].value_counts()


# replace Unavailable/Unknown with Unknown
df_transformed['preclinicalresultsstatus'] = df_transformed['preclinicalresultsstatus'].replace({909670004.0 : 909670002.0})



In [234]:
# transform catagories in approval status to one less.
df_transformed['approvalstatus'] = df_transformed['approvalstatus'].replace({862890001 :909670000})

In [235]:
status_counts = df[('new_2024currentrdstage')].value_counts()
#print(status_counts)

df_transformed['2024rdstage'] = df_transformed['2024rdstage'].replace({
            'Late development (design and development)' :'Late development',
            'Late development (clinical validation and launch readiness)' : 'Late development',
            'Phase III - Drugs' : 'Phase III',
            'Not applicable' : 'N/A',
            'Discovery' : 'Discovery and Preclinical',
            'Early development (concept and research)' : 'Early development',
            'Early development (feasibility and planning)' : 'Early development',
        'Late development (clinical validation and launch readiness) - Diagnostics' : 'Late development',
            'Late development - Diagnostics' : 'Late development',
            'Phase II - Vaccines' : 'Phase II',
    'Phase III - Vaccines' : 'Phase III',
    'Phase IV - Vaccines' : 'Phase IV',
    'Phase I - Vaccines' : 'Phase I',
   'Preclinical - Vaccines' : 'Preclinical',
    'Regulatory filing - Diagnostics' : 'Regulatory filing',
    'Preclinical - Drugs': 'Preclinical'

})


In [236]:
status_counts = df[('vin_approvingauthority')].value_counts()
df_transformed['approvingauthority'] = df_transformed['approvingauthority'].replace({
    909670002 : 909670001
})


vin_approvingauthority
909670000    255
909670001    217
909670003    160
909670002      3
Name: count, dtype: int64


In [237]:
#include only candidates and products of interest
#100000002 = pending
#100000000 = yes
df_final = df_transformed[(df_transformed['includeinpipeline'] == 100000000) | (df_transformed['includeinpipeline'] == 100000002)]

In [238]:
df_transformed.shape

(9383, 94)

In [239]:
df_final.shape

(4045, 94)

### change the order of the columns

In [240]:
#get list with all columns
cols =df_final.columns.tolist()

print(cols)

['candidate_name', 'sbereviewstatus', 'targettoxinclass', 'knownfundersaggregated', 'whosnakespeciesriskcatagainst', 'approvalstatus', 'venomspecificity', 'pressuretype', 'ctregistrylink', 'modifiedon', 'approvedforuseinpregnantorlactatingwomen', 'technologyprinciple', 'ctenddate', '2019developers', 'usfdaapprovaldate', 'alternativenames', 'product', 'researchedinpregnantwomenorlactatingwomen', 'target', 'emaapprovalstatus', 'inactivedevelopmentreason', '2024knownfunders', 'emaapprovaldate', 'previouslyidentifiedcandidate', '2023knownfunders', 'WH_indication', 'developersaggregated', 'japanesemhlwapprovalstatus', 'typeofpreclinicalresults', 'vin_patent', 'vin_highestrdstage', 'new_targetcountry', 'SRA_approvalstatus', 'whoprequalificationdate', 'technologytype', 'snakespeciesagainst', 'numberofcountrieswithproductapproval', 'vin_investigatedforotherindications', 'statecode', 'new_snakespeciesproducttestedin', 'new_targetpopulation', 'new_productconcept', 'vin_candidateno', 'vin_ifmarke

In [244]:
columns_ordered = [
    'candidate_name', 'alternativenames', 'diseasevalue', 'secondarydisease_value',
    'product', 'technologyprinciple', 'technologytype', 'target',
    'inactivedevelopmentreason', 'previouslyidentifiedcandidate', 'vin_patent',
    'new_targetpopulation', 'new_productconcept', 'vin_candidateno', 'indicationtype',
    'indication', 'chemicalname', 'mechanismofaction', 'anticipatedwhostrategy',
    'healthcarefacilitylevel', 'mainproduct_value', 'subproduct_value',
    'keyfeatureschallenges', 'agespecific', 'developmentstatus',
    'clinicalusestatus_value', 'preclinicalresultssource', 'preclinicalresultsstatus',
    'vin_highestrdstage', 'new_rdstage2023', '2023currentrdstage', '2024rdstage',
    'rdstage_value', 'approvingauthority', 'approvalstatus', 'usfdaapprovaldate',
    'emaapprovalstatus', 'emaapprovaldate', 'japanesemhlwapprovaldate',
    'japanesemhlwapprovalstatus', 'SRA_approvalstatus', 'whoprequalificationdate',
    'whoprequalification', 'numberofcountrieswithproductapproval',
    'vin_ifmarketedregulatoryapprovaltypelevel', 'vin_otherstringentregulatoryauthorityapproval',
    'vin_numberofcountrieswithproductapproval_state', 'usfdaapprovalstatus',
    'otherSRAdate', 'NRAapprovalstatus', 'countries_product_approved',
    'developersaggregated', '2019developers', '2023developersaggregated',
    '2024developersaggregated', '2024developmentstatus', '2023developmentstatus',
    'knownfundersaggregated', '2024knownfunders', '2023knownfunders',
    'ctregistrylink', 'ctenddate', 'typeofpreclinicalresults',
    'vin_investigatedforotherindications', 'ctstartdate', 'sbereviewstatus',
    'targettoxinclass', 'whosnakespeciesriskcatagainst', 'venomspecificity',
    'pressuretype', 'new_targetcountry', 'snakespeciesagainst',
    'new_snakespeciesproducttestedin', 'new_snakefamilyagainst', 'snakespecies',
    'specifictargettoxinclass', 'specimentype', 'snakespeciesproductderivedfrom',
    'whoimmunizingspecies', 'snakefamily', 'whosnakespeciesriskcategory',
    'whoparaspecificityspecies', 'thermostability', 'approvedforuseinpregnantorlactatingwomen',
    'researchedinpregnantwomenorlactatingwomen', 'WH_indication', 'peseverity',
    'modifiedon', 'statecode', 'recentupdates', 'candidateid',
    'includeinpipeline', 'valid_from', 'valid_to'
]

In [245]:
len(columns_ordered)

94

In [246]:
df_final = df_final[columns_ordered]

In [247]:
df_final

,candidate_name,alternativenames,diseasevalue,secondarydisease_value,product,technologyprinciple,technologytype,target,inactivedevelopmentreason,previouslyidentifiedcandidate,...,researchedinpregnantwomenorlactatingwomen,WH_indication,peseverity,modifiedon,statecode,recentupdates,candidateid,includeinpipeline,valid_from,valid_to
versionnumber,,,,,,,,,,,,,,,,,,,,,
110768105,DPP Fever Panel II Asia IgM,None,90b20daf-d5a0-ec11-b400-002248185df1,None,Diagnostics,100000005.0,Immunoassay,IgM Antibody,None,NaN,...,NaN,None,NaN,2025-12-17T17:48:00Z,0,MH 13/08/2025: 2024 - Diagnostic accuracy of t...,5582e435-4f77-f011-b4cc-000d3a6a2e50,100000000.0,2025-12-17T17:48:00Z,None
110768107,TRURAPID MPXV Ag Test,None,27351f82-7d6e-ef11-a670-0022489309da,None,Diagnostics,100000005.0,Immunoassay,Monkeypox virus antigen,None,NaN,...,NaN,None,NaN,2025-12-17T17:48:00Z,0,None,341c9ffa-5577-f011-b4cc-000d3a6a2e50,100000000.0,2025-12-17T17:48:00Z,None
110768091,NABIT Mpox Test,None,27351f82-7d6e-ef11-a670-0022489309da,None,Diagnostics,100000007.0,Molecular,Monkeypox virus DNA,None,NaN,...,NaN,None,NaN,2025-12-17T17:48:00Z,0,None,5b719066-5877-f011-b4cc-000d3a6a2e50,100000000.0,2025-12-17T17:48:00Z,None
110768093,MPV Ag Rapid Test,None,27351f82-7d6e-ef11-a670-0022489309da,None,Diagnostics,100000005.0,Immunoassay,Monkeypox virus antigen,None,NaN,...,NaN,None,NaN,2025-12-17T17:48:00Z,0,None,7fbc37f8-5977-f011-b4cc-000d3a6a2e50,100000000.0,2025-12-17T17:48:00Z,None
110768109,SGTi-flex Mpox Ag,None,27351f82-7d6e-ef11-a670-0022489309da,None,Diagnostics,100000001.0,Immunoassay,Monkeypox virus specific antigen,None,NaN,...,NaN,None,NaN,2025-12-17T17:48:00Z,0,None,8bc1dbc3-5b77-f011-b4cc-000d3a6a2e50,100000000.0,2025-12-17T17:48:00Z,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110776575,Chromosomal Microarray Analysis (CMA),None,683d7060-1f0b-ee11-8f6d-6045bd3df63a,None,Diagnostics,NaN,None,N/A,None,NaN,...,NaN,None,NaN,2025-12-18T01:47:41Z,0,None,e7ae56e4-d4da-f011-8544-7ced8da2b46a,100000000.0,2025-12-18T01:47:41Z,None
110776573,Angiogenic and endothelial biomarkers (serum v...,None,683d7060-1f0b-ee11-8f6d-6045bd3df63a,None,Diagnostics,NaN,None,N/A,None,NaN,...,NaN,None,NaN,2025-12-18T01:47:41Z,0,None,c29db9fe-d7da-f011-8544-7ced8da2b46a,100000000.0,2025-12-18T01:47:41Z,None
110558739,Audenz,None,c5ad3b90-7372-f011-b4cd-000d3a6a2e50,None,Vaccines,NaN,Inactivated,Influenza virus surface antigens: A/turkey/Tur...,None,NaN,...,NaN,None,NaN,2025-12-15T23:54:47Z,0,None,ef05fbd5-1eab-f011-bbd2-7ced8da2d233,100000000.0,2025-12-15T23:54:47Z,None
